In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.utils import shuffle
import pandas as pd
import pickle
from matplotlib.pyplot import MultipleLocator

In [ ]:
resolver = tf.distribute.cluster_resolver.TPUClusterResolver(tpu='')
tf.config.experimental_connect_to_cluster(resolver)
# This is the TPU initialization code that has to be at the beginning.
tf.tpu.experimental.initialize_tpu_system(resolver)
print("All devices: ", tf.config.list_logical_devices('TPU'))

In [ ]:
strategy = tf.distribute.TPUStrategy(resolver)

In [ ]:
train_data = pd.read_csv(r'../input/blood-pressure-datasets/Train_Merge_Data.csv')
train_dev = pd.read_csv(r'../input/ppg-dev/train_dev.csv')

validation_data = pd.read_csv(r'../input/blood-pressure-datasets/Validation_Merge_Data.csv')
validation_dev = pd.read_csv(r'../input/ppg-dev/validation_dev.csv')

In [ ]:
train_data = pd.concat([train_data,train_dev],axis=1)
validation_data = pd.concat([validation_data,validation_dev],axis=1)

In [ ]:
train_data = shuffle(train_data)
validation_data = shuffle(validation_data)

In [ ]:
train_label_Sbp = train_data.iloc[:,610]
train_label_Sbp = train_label_Sbp.values

train_label_Dbp = train_data.iloc[:,611]
train_label_Dbp = train_label_Dbp.values

train_dev1 = train_data.iloc[:,662:1262]
train_dev1 = train_dev1.values

train_dev2 = train_data.iloc[:,1262:]
train_dev2 = train_dev2.values

train_data = train_data.iloc[:,:600]
train_data = train_data.values

In [ ]:
validation_label_Sbp = validation_data.iloc[:,610]
validation_label_Sbp = validation_label_Sbp.values

validation_label_Dbp = validation_data.iloc[:,611]
validation_label_Dbp = validation_label_Dbp.values

validation_dev1 = validation_data.iloc[:,662:1262]
validation_dev1 = validation_dev1.values

validation_dev2 = validation_data.iloc[:,1262:]
validation_dev2 = validation_dev2.values

validation_data = validation_data.iloc[:,:600]
validation_data = validation_data.values

In [ ]:
print("train_data_information:")
print(train_data.shape)
print(train_dev1.shape)
print(train_dev2.shape)
print(train_label_Sbp.shape)
print(train_label_Dbp.shape)
print("validation_data_information:")
print(validation_data.shape)
print(validation_dev1.shape)
print(validation_dev2.shape)
print(validation_label_Sbp.shape)
print(validation_label_Dbp.shape)

In [ ]:
#---------------------------------------#
#   激活函数 relu6
#---------------------------------------#
def relu6(x):
    return tf.keras.activations.relu(x, max_value=6)

#---------------------------------------#
#   利用relu函数乘上x模拟sigmoid
#---------------------------------------#
def hard_swish(x):
    return x * tf.keras.activations.relu(x + 3.0, max_value=6.0) / 6.0

#---------------------------------------#
#   通道注意力机制单元
#   利用两次全连接算出每个通道的比重
#   可以连接在任意特征层后面
#---------------------------------------#
def squeeze(inputs):
    input_channels = int(inputs.shape[-1])
    
    x = layers.GlobalAveragePooling1D()(inputs)

    x = layers.Dense(int(input_channels/4))(x)
    x = relu6(x)

    x = layers.Dense(input_channels)(x)
    x = hard_swish(x)

    x = layers.Reshape((1, input_channels))(x)
    #print(x)
    #print(inputs)
    x = layers.Multiply()([inputs, x])
    return x

#基础层下采样
def DownSampling(inputs,filters):
    layer = layers.Conv1D(filters,3,padding='same')(inputs)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    pool = layers.MaxPooling1D(pool_size=2)(layer)
    
    return layer,pool

#基础层上采样
def UpSampling(inputs,con_input,filters,need_zero=False):
    
    
    up_layer = layers.UpSampling1D(size=2)(inputs)
    
    if need_zero==True:
        up_layer = layers.ZeroPadding1D((0,1))(up_layer)
    
    layer = layers.Conv1D(filters,2,padding='same')(up_layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)
    
    layer = layers.Concatenate(axis=2)([layer,con_input])
    
    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    layer = layers.Activation(tf.nn.relu)(layer)

    layer = layers.Conv1D(filters,3,padding='same')(layer)
    layer = layers.BatchNormalization()(layer)
    output = layers.Activation(tf.nn.relu)(layer)
    
    return output


In [ ]:
def create_model():
    #双输入模型
    inputs_1 = keras.Input(shape=(600,1),name='inputs_1')
    inputs_dev1 = keras.Input(shape=(600,1),name='inputs_dev1')
    inputs_dev2 = keras.Input(shape=(600,1),name='inputs_dev2')
    
    inputs = layers.Concatenate(axis=2)([inputs_1,inputs_dev1,inputs_dev2])
    #下采样
    layer1,pool1 = DownSampling(inputs,filters=64)
    
    
    layer2,pool2 = DownSampling(pool1,filters=128)
    
        
    layer3,pool3 = DownSampling(pool2,filters=256)
    
    
    layer4,pool4 = DownSampling(pool3,filters=512)
    
    
    #最后一层不需要Pooling
    layer5,pool5 = DownSampling(pool4,filters=1024)
   
    #上采样
    
    #attention
    layer4 = squeeze(layer4)
    
    layer44 = UpSampling(layer5,layer4,512,need_zero=True)
    
    #attention
    layer3 = squeeze(layer3)
                     
    layer33 = UpSampling(layer44,layer3,256)
    
    #attention
    layer2 = squeeze(layer2)
    
    layer22 = UpSampling(layer33,layer2,128)
    
    #attention
    layer1 = squeeze(layer1)
    
    layer11 = UpSampling(layer22,layer1,64)

    layer = layers.GlobalAveragePooling1D()(layer11)

    outputs_sbp = layers.Dense(1,name='Sbp')(layer)
    outputs_dbp = layers.Dense(1,name='Dbp')(layer)

    model = keras.Model(inputs=[inputs_1,inputs_dev1,inputs_dev2],outputs=[outputs_sbp,outputs_dbp])
    
    return model

In [ ]:
"""自定义评价指标模块"""
def standard_deviation(y_true, y_pred):
    u = keras.backend.mean(y_pred-y_true)
    return keras.backend.sqrt(keras.backend.mean(keras.backend.square((y_pred-y_true) - u)))

In [ ]:
"""回调函数"""
#保存迭代周期内最好的模型
checkpoint_filepath = r'./model_struction.h5'
Save_epochs = 10 #迭代多少层保存一次模型
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    # save_weights_only=True,
    monitor='val_Sbp_mean_absolute_error',
    mode='min',
    save_best_only=True)

In [ ]:
with strategy.scope():
    model = create_model()
    
    model.compile(loss={'Sbp':"mse",'Dbp':"mse"}, optimizer=keras.optimizers.Adam(lr=0.0001),metrics=[tf.keras.metrics.MeanAbsoluteError(),standard_deviation])

In [ ]:
#model = create_model()

In [ ]:
model.summary()

In [ ]:
"""保存模型结构图片"""
tf.keras.utils.plot_model(model, to_file=r'./model_graph.png', show_shapes=True, show_layer_names=False)

In [ ]:
history = model.fit({'inputs_1':train_data,'inputs_dev1':train_dev1,'inputs_dev2':train_dev2},{'Sbp':train_label_Sbp,'Dbp':train_label_Dbp},
                    batch_size=128*8,
                    epochs=500,
                    callbacks=model_checkpoint_callback,
                    validation_data=({'inputs_1':validation_data,'inputs_dev1':validation_dev1,'inputs_dev2':validation_dev2},{'Sbp':validation_label_Sbp,'Dbp':validation_label_Dbp})
                    )

In [ ]:
with open(r'./last_model.h5.pickle', 'wb') as file_pi:
 	pickle.dump(history.history, file_pi)